In [1]:
from __future__ import division
import numpy as np
import scipy as sc
from itertools import product
import time
import matplotlib.pyplot as plt
import PIL
from numpy import log10
import random
from math import factorial
from scipy.stats import linregress, gaussian_kde, skew
from scipy import stats
from scipy.spatial import distance
import warnings
import pandas as pd
import re
import os
import math
from collections import Counter
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.stats.outliers_influence import summary_table
import statsmodels.api as sm
from scipy.optimize import linear_sum_assignment

warnings.filterwarnings('ignore')

%config InlineBackend.figure_formats = ['svg']
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

pd.set_option('display.max_columns', None)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


# Figure 2.

## Analyzing species rarity
 
The analysis of shift is highly relevant to the study of species-abundance distributions (SADs) (Fig 8a) [10, 12, 14, 21]. These histograms of species abundance underpin thousands of ecological studies spanning all domains of life and major habitats [10, 12, 14, 21]. SADs are often compared to theoretical predictions, to SADs sampled from other communities of similar taxa, and are the basis for many measures of species dominance, diversity, evenness, and rarity [9, 10, 12, 14, 21]. Consequently, the field of ecology is replete with techniques for analyzing SADs and for quantifying the aspects of biodiversity they contain.


In considering that RDS provides a means of quantifying the magnitude and direction by which one SAD is concentrated to lesser or greater abundances relative to another SAD,  

Beyond providing yet another means of comparing SADs, DS and RDS provide an intuitive and easily interpretable means of quantifying species rarity, i.e., the concentration of species at low abundances [14, 21]. 

However, 

As the concentration of frequencies away from the discrete class having the greatest value, DS is highly similar in concept to species rarity. Consequently, given its bounded values and intuitive interpretation, we asked whether DS provides a preferrable measure of species rarity. in particular, because skewness based measures are not bounded an

Unlike skewness-based measures, RDS is bounded, intuitive, and directly reflects the concentration of species towards the class of lowest abundance. Likewise, RDS represents a better comparative metric of species rarity for the same reasons.

In [2]:
def count_pts_within_radius(x, y, radius, scale=0):
    """Count the number of points within a fixed radius in 2D space"""
    
    raw_data = np.array([x, y])
    x = np.array(x)
    y = np.array(y)
    raw_data = raw_data.transpose()
    
    # Get unique data points by adding each pair of points to a set
    unique_points = set()
    for xval, yval in raw_data:
        unique_points.add((xval, yval))
    
    count_data = []
    for a, b in unique_points:
        if scale == 'sqrt':
            num_neighbors = len(x[((sqrt(x) - sqrt(a)) ** 2 +
                                   (sqrt(y) - sqrt(b)) ** 2) <= sqrt(radius) ** 2])
        else:        
            num_neighbors = len(x[((x - a) ** 2 + (y - b) ** 2) <= radius ** 2])
        count_data.append((a, b, num_neighbors))
    return count_data



def plot_color_by_pt_dens(x, y, radius, loglog=0, scale=0, plot_obj=None, point_size=10):
    
    """Plot bivariate relationships with large n using color for point density

    Inputs:
    x & y -- variables to be plotted
    radius -- the linear distance within which to count points as neighbors
    scale -- a flag to indicate the use of a scale plot (scale = 1)

    The color of each point in the plot is determined by the logarithm (base 10)
    of the number of points that occur with a given radius of the focal point,
    with hotter colors indicating more points. The number of neighboring points
    is determined in linear space regardless of whether a scale plot is
    presented.
    """
    
    plot_data = count_pts_within_radius(x, y, radius, scale)
    sorted_plot_data = np.array(sorted(plot_data, key=lambda point: point[2]))

    if plot_obj == None:
        plot_obj = plt.axes()
        
    plot_obj.scatter(sorted_plot_data[:, 0],
            sorted_plot_data[:, 1],
            facecolors='none',
            s = point_size, 
            edgecolors='0.1', 
            linewidths=1., 
            #cmap='Greys'
            )
    
    # plot points
    c = np.array(sorted_plot_data[:, 2])**0.25
    c = np.max(c) - c
    plot_obj.scatter(sorted_plot_data[:, 0],
                    sorted_plot_data[:, 1],
                    c = c,
                    s = point_size, 
                    edgecolors='k', 
                    linewidths=0.0, 
                    cmap='Greys_r',
                    #alpha = 0.5,
                    )
        
    return plot_obj


In [3]:

def DS(p):
    p_bins = len(p)
    p_obs = sum(p)
    
    z_p = (p_bins + 1)/p_bins
    p = [sum(p[:ii+1])**(z_p) for ii in range(len(p))]
    Sp = np.sum(np.array(p)/(p_obs**z_p)) - 1
    ds = Sp/(p_bins - 1)
    return ds


In [4]:
def Rlogskew(sad):
    '''
    Calculation of rarity used in:
        A.E. Magurran and B.J. McGill, eds. Biological diversity: frontiers in measurement and assessment. 
        OUP Oxford, 2010.

    '''
    
    S = len(sad)

    if S <= 2.0:
        print('S < 2, cannot compute log-skew')
        sys.exit()

    sad = np.log10(sad)
    mu = np.mean(sad)

    num = 0
    denom = 0
    for ni in sad:
        num += ((ni - mu)**3.0)/S
        denom += ((ni - mu)**2.0)/S

    t1 = num/(denom**(3.0/2.0))
    t2 = (S/(S - 2.0)) * np.sqrt((S - 1.0)/S)

    return t1 * t2


def get_RADs(path, name, closedref=True):

    # Get rank-abundance distributions, i.e, abundance vectors
    
    RADdict = {}
    DATA = path + name + '-data.txt'

    with open(DATA) as f:

        for d in f:

            if d.strip():
                d = d.split()
                length = len(d)

                if name == 'GENTRY':
                    site = d[0]
                    #species = d[1] # Dataset name plus species identifier
                    abundance = float(d[-1])

                else:
                    site = d[0]
                    #year = d[1]

                    if closedref == True:
                        for i in d:
                            if 'unclassified' in i:
                                #print('unclassified')
                                continue
                            elif 'unidentified' in i:
                                #print('unidentified')
                                continue

                    abundance = float(d[-1])


                if abundance > 0:
                    if site in RADdict:
                        RADdict[site].append(abundance)
                    else:
                        RADdict[site] = [abundance]

    RADs = RADdict.values()
    filteredRADs = []
    for rad in RADs:
        if len(rad) >= 10:
            filteredRADs.append(rad)

    return filteredRADs



def EMP_RADs(path, name):

    minS = 10

    IN = path + '/EMPclosed-SADs.txt'
    rads = []
    
    with open(IN) as f:

        for rad in f:
            rad = eval(rad)
            if len(rad) >= minS:
                rads.append(rad)

    return rads

def Louca_RADs(path, name):
    
    # Get rank-abundance distributions, i.e, abundance vectors
    
    RADdict = {}
    DATA = path + 'SSADdata.txt'

    with open(DATA) as f:

        for d in f:

            if d.strip():
                d = d.split()
                length = len(d)

                site = d[1]
                for i in d:
                    abundance = float(d[-1])


                if abundance > 0:
                    if site in RADdict:
                        RADdict[site].append(abundance)
                    else:
                        RADdict[site] = [abundance]

    RADs = RADdict.values()
    filteredRADs = []
    for rad in RADs:
        if len(rad) >= 10:
            filteredRADs.append(rad)

    return filteredRADs


def NSECF(p):
    p_bins = len(p)
    p_obs = sum(p)
    
    z_p = (p_bins + 1)/p_bins
    p = [sum(p[:ii+1])**(z_p) for ii in range(len(p))]
    return np.sum(np.array(p)/(p_obs**z_p)) - 1
    
                
def getMetrics():
    
    name_ls = []
    kind_ls = []
    N_ls = []
    S_ls = []
    skew_ls = []
    logskew_ls = []
    log_mod_skew_ls = []
    log_mod_skew_log_ls = []
    ds_ls = []
    nsecf_ls = []
    
    datasets = []
    for name in os.listdir('data/ecological/micro'):
            datasets.append([name, 'micro'])
    for name in os.listdir('data/ecological/macro'):
            datasets.append([name, 'macro'])

    for dataset in datasets:
        
        name = dataset[0] # name of dataset
        kind = dataset[1] # micro or macro

        if name == '.DS_Store' or name == 'MGRAST': 
            continue
        
        OUT = open('data/ecological/'+kind+'/'+name+'/'+name+'-SADMetricData.txt','w+')
        RADs = []

        if kind == 'macro':
            RADs = get_RADs('data/ecological/'+kind+'/'+name+'/', name)
            print('macro', name, len(RADs))


        if kind == 'micro':
            if name == 'EMPclosed' or name == 'EMPopen':
                RADs = EMP_RADs('data/ecological/'+kind+'/'+name+'/', name)
            
            elif name == 'Louca':
                RADs = Louca_RADs('data/ecological/'+kind+'/'+name+'/', name)
            
            else:
                RADs = get_RADs('data/ecological/'+kind+'/'+name+'/', name)

            print('micro', name, len(RADs))

        ct = 0
        numRADs = len(RADs)
        for RAD in RADs:

            if kind == 'micro':
                RAD = list([x for x in RAD if x > 0])

            elif kind == 'macro':
                RAD = list([x for x in RAD if x > 0])


            N = sum(RAD)
            S = len(RAD)

            if S < 10: 
                continue
            if max(RAD) == min(RAD): 
                continue

            # Measures of Rarity
            
            # 1. skewness of abundances
            skewness = skew(RAD)
            
            # 2. log-modulo transformation of skewnness
            lms = np.log10(np.abs(float(skewness)) + 1)
            if skewness < 0: 
                lms = lms * -1
            log_mod_skew = float(lms)
            
            # 3. skewness of log-transformed abundances
            logskew = Rlogskew(RAD)
            
            # 4. log-modulo transformation of skewnness of log-transformed abundances
            lms = np.log10(np.abs(float(logskew)) + 1)
            if skewness < 0: 
                lms = lms * -1
            log_mod_skew_log = float(lms)
            
            # 5. Distributional shift (DS)
            # Convert the abundances to logarithmic scale (base 2)
            abundances = np.log2(RAD)

            # Define the bins for the histogram
            min_abundance = 0 #np.floor(min(abundances))
            max_abundance = np.ceil(max(abundances))
            bins = np.arange(min_abundance, max_abundance + 1, 1)

            # Compute the histogram
            hist, bin_edges = np.histogram(abundances, bins=bins)

            # Use the right side of the bin edges as bin values
            bin_values = bin_edges[1:]

            # Convert histogram to list
            bin_heights = hist.tolist()
            ds = DS(bin_heights)
            
            # 6. Normalized sums of exponentiated cumulative frequencies
            nsecf = NSECF(bin_heights)

            ct+=1

            print(name, kind, N, S, skewness, logskew, log_mod_skew, log_mod_skew_log, ds, nsecf, file=OUT)
            
            name_ls.append(name)
            kind_ls.append(kind)
            N_ls.append(N)
            S_ls.append(S)
            skew_ls.append(skewness)
            logskew_ls.append(logskew)
            log_mod_skew_ls.append(log_mod_skew)
            log_mod_skew_log_ls.append(log_mod_skew_log)
            ds_ls.append(ds)
            nsecf_ls.append(nsecf)
            
        OUT.close()
        
    return name_ls, kind_ls, N_ls, S_ls, skew_ls, logskew_ls, log_mod_skew_ls, log_mod_skew_log_ls, ds_ls, nsecf_ls


In [5]:
name_ls, kind_ls, N_ls, S_ls, skew_ls, logskew_ls, log_mod_skew_ls, log_mod_skew_log_ls, ds_ls, nsecf_ls = getMetrics()

main_df = pd.DataFrame({
    'name': name_ls,
    'kind': kind_ls,
    'N': N_ls,
    'S': S_ls,
    'skew': skew_ls,
    'logskew': logskew_ls,
    'log_mod_skew': log_mod_skew_ls,
    'log_mod_skew_log': log_mod_skew_log_ls,
    'DS': ds_ls,
    'NSECF': nsecf_ls,
})

print(main_df.shape)

print(main_df['name'].unique().tolist())
main_df.head()

micro CHU 44
micro CATLIN 130
micro BOVINE 16
micro LAUB 84
micro SED 34
micro BIGN 13
micro CHINA 186
micro TARA 139
micro HMP 4303
micro FUNGI 128
micro Louca 8308
micro HUMAN 525
micro HYDRO 123
micro EMPclosed 14631
macro FIA 10355
macro CBC 1999
macro MCDB 103
macro BBS 2769
macro GENTRY 222
(44090, 10)
['CHU', 'CATLIN', 'BOVINE', 'LAUB', 'SED', 'BIGN', 'CHINA', 'TARA', 'HMP', 'FUNGI', 'Louca', 'HUMAN', 'HYDRO', 'EMPclosed', 'FIA', 'CBC', 'MCDB', 'BBS', 'GENTRY']


,name,kind,N,S,skew,logskew,log_mod_skew,log_mod_skew_log,DS,NSECF
0,CHU,micro,2487.0,350,11.376136,1.415601,1.092585,0.383025,0.839037,6.712296
1,CHU,micro,2032.0,306,9.868336,1.355590,1.036163,0.372100,0.832462,6.659695
2,CHU,micro,872.0,181,4.654126,1.179171,0.752365,0.338291,0.801513,4.809080
3,CHU,micro,1241.0,202,9.932251,1.676968,1.038710,0.427643,0.861797,6.894373
4,CHU,micro,785.0,174,7.223195,1.498230,0.915041,0.397632,0.815210,4.891259


In [6]:
print(main_df.shape)

its = 20
tdf = None
names = ['CHU', 'CATLIN', 'BOVINE', 'LAUB', 'SED', 'BIGN', 
         'CHINA', 'TARA', 'HMP', 'FUNGI', 'Louca', 'HUMAN', 
         'HYDRO', 'EMPclosed', 'FIA', 'CBC', 'MCDB', 'BBS', 
         'GENTRY']

for n in range(its):

    for i, name in enumerate(names):
                
        tdf_nm = main_df[main_df['name'] == name]
        kind = tdf_nm['kind'].iloc[0]
        numlines = tdf_nm.shape[0]
                
        small = ['BIGN', 'BOVINE', 'CHU', 'LAUB', 'SED']
        big = ['HUMAN', 'CHINA', 'CATLIN', 'FUNGI', 'HYDRO']

        if name == 'Louca':
            tdf_nm = tdf_nm.sample(1000, replace=True)

        elif kind == 'macro':
            tdf_nm = tdf_nm.sample(100, replace=True)
        elif name in small:
            tdf_nm = tdf_nm.sample(20, replace=True)
        elif name in big:
            tdf_nm = tdf_nm.sample(50, replace=True)
        elif name == 'TARA':
            tdf_nm = tdf_nm.sample(50, replace=True)
        else:
            tdf_nm = tdf_nm.sample(50, replace=True)
            
        if n == 0 and i == 0:
            tdf = tdf_nm.copy(deep=True)
        else:
            tdf = pd.concat([tdf, tdf_nm])
            

(44090, 10)


In [7]:

tdf.dropna(subset=['DS'], how='any', inplace=True)

# Log scale the 'N' values
log_N = np.log10(tdf['N'])

# log_mod_skew values
log_mod_skew = tdf['log_mod_skew']

# DS values
DS_vals = tdf['DS']

# NSECF
NSECF_vals = tdf['NSECF']

# Reshape the data to fit the model
X = log_N.values
y1 = log_mod_skew.values
y2 = DS_vals.values
y3 = NSECF_vals.values

# Fit the linear regression model for log_mod_skew
slope1, intercept1, r_value1, p1, se1 = linregress(X, y1)
print("Regression for log_mod_skew:")
print("Slope:", slope1)
print("Intercept:", intercept1)
print("r2:", r_value1 ** 2, '\n')

# Fit the linear regression model for DS
slope2, intercept2, r_value2, p2, se2 = linregress(X, y2)
print("Regression for DS:")
print("Slope:", slope2)
print("Intercept:", intercept2)
print("r2:", r_value2 ** 2, '\n')

# Fit the linear regression model for NSECF
slope3, intercept3, r_value3, p3, se3 = linregress(X, y3)
print("Regression for NSECF:")
print("Slope:", slope3)
print("Intercept:", intercept3)
print("r2:", r_value3 ** 2)

# Plotting
fig = plt.figure(figsize=(10, 2.5))
x_lab = 'log(N)'

# Scatter plot and regression line for log_mod_skew
plot_color_by_pt_dens(X, y1, radius=0.05, loglog=0, plot_obj=plt.subplot(1, 3, 1), point_size=10)
slope, intercept, r_val, p_val, std_err = linregress(X, y1)
fitted_vals = slope * np.array(X) + intercept
s = r'$r_{2}$' + ' = ' + str(round(r_val**2, 2))
plt.plot(X, fitted_vals, color='k', linewidth=2, label=s)
plt.xlabel(x_lab, fontweight='bold')
plt.ylabel('log-modulo skewness', fontweight='bold')
plt.title(r'Rarity = '+str(round(10**intercept,2))+'*'+r'$N$'+'$^{'+str(round(slope,2))+'}$')
plt.legend(frameon=False)

# Scatter plot and regression line for DS (non-standardized)
plot_color_by_pt_dens(X, y3, radius=0.05, loglog=0, plot_obj=plt.subplot(1, 3, 2), point_size=10)
slope, intercept, r_val, p_val, std_err = linregress(X, y3)
fitted_vals = slope * np.array(X) + intercept
s = r'$r_{2}$' + ' = ' + str(round(r_val**2, 2))
plt.plot(X, fitted_vals, color='k', linewidth=2, label=s)
plt.xlabel(x_lab, fontweight='bold')
plt.ylabel('DS, non-standardized', fontweight='bold')
plt.title(r'Rarity = '+str(round(slope,2))+'*'+r'$N$'+ ' - ' + str(abs(round(intercept,2))))
plt.legend(frameon=False)


# Scatter plot and regression line for DS
plot_color_by_pt_dens(X, y2, radius=0.05, loglog=0, plot_obj=plt.subplot(1, 3, 3), point_size=10)
slope, intercept, r_val, p_val, std_err = linregress(X, y2)
fitted_vals = slope * np.array(X) + intercept
s = r'$r_{2}$' + ' = ' + str(round(r_val**2, 2))
plt.plot(X, fitted_vals, color='k', linewidth=2, label=s)
plt.xlabel(x_lab, fontweight='bold')
plt.ylabel('DS, standardized', fontweight='bold')
plt.legend(frameon=False)


fig.patch.set_facecolor('white')
plt.subplots_adjust(wspace=0.45, hspace=0.35)
plt.savefig('Final_Figs/manuscript/Fig2.pdf', bbox_inches='tight', format='pdf', dpi=600)
plt.savefig('Final_Figs/manuscript/Fig2.jpg', bbox_inches='tight', format='jpg', dpi=600)
plt.close()


Regression for log_mod_skew:
Slope: 0.18601026595889422
Intercept: 0.13896086999748047
r2: 0.5922371138946283 

Regression for DS:
Slope: 0.027741989515227494
Intercept: 0.602322091588741
r2: 0.06779993014824855 

Regression for NSECF:
Slope: 2.405479534031236
Intercept: -1.8782641482141953
r2: 0.8477463486565046
